# Notebook - Data Augmentation

In [1]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

In [51]:
data_df = pd.read_parquet("../dataset/train_df_all_english.parquet")

In [52]:
data_df.groupby("impact_length_idx").head(5).sort_values("impact_length_idx")

,sentence,Translation,impact_length_idx,language
4,"EU Regulators Welcome, Critique New European S...","EU Regulators Welcome, Critique New European S...",0.0,English
16,Eni Doubles Sustainability-Linked Bond Offerin...,Eni Doubles Sustainability-Linked Bond Offerin...,0.0,English
19,Bank of America CEO: Sustainable Finance Deman...,Bank of America CEO: Sustainable Finance Deman...,0.0,English
21,"Republican Politicians Target Proxy Firms ISS,...","Republican Politicians Target Proxy Firms ISS,...",0.0,English
22,Guest Post: ESG isn’t About Altruism – it’s Ab...,Guest Post: ESG isn’t About Altruism – it’s Ab...,0.0,English
0,Arabesque AI Appoints Carolina Minio Paluello ...,Arabesque AI Appoints Carolina Minio Paluello ...,1.0,English
1,Arabesque AI Appoints Carolina Minio Paluello ...,Arabesque AI Appoints Carolina Minio Paluello ...,1.0,English
2,Arabesque AI Appoints Carolina Minio Paluello ...,Arabesque AI Appoints Carolina Minio Paluello ...,1.0,English
7,Red States Sue to Stop Biden Administration Ru...,Red States Sue to Stop Biden Administration Ru...,1.0,English
11,Methane Emissions Detection Platform Kuva Rais...,Methane Emissions Detection Platform Kuva Rais...,1.0,English


In [53]:
data_augmentation_dict = {
    "less than 2 years": [data_df.loc[4]["Translation"], data_df.loc[19]["Translation"]],
    "between 2 and 5 years": [data_df.loc[7]["Translation"], data_df.loc[11]["Translation"]],
    "more than 5 years": [data_df.loc[6]["Translation"], data_df.loc[8]["Translation"]]
}

In [55]:
data_augmentation_messages = []

repeat_n = 2 # call api 2 more times

for impact_duration, text_list in data_augmentation_dict.items():

    template = f"""Give 10 examples of real news related to ESG (Environmental, Social, Governance) that has an impact duration {impact_duration}. Each example should have a news title and news summary, connected with "||". Generate these examples in english language. Two example news:

    "{text_list[0]}"
    
    "{text_list[1]}"
    
    The output should only contain a numbered list of news.
    
    """
    
    conversation = [
            {"role": "system", "content": "You are an ESG analyst, skilled assessing the level and duration an event in the news article might have on the company"},
            {"role": "user", "content": template}
        ]
    
    print(template)
    
    # completion = client.chat.completions.create(
    #     model="gpt-4",
    #     messages=conversation
    # )
    
    # message = completion.choices[0].message.content
    
    # print(message)
    
    # data_augmentation_messages.append(message)
    
    # for i in range(repeat_n):
    #     assitant_message = {"role": "assistant", "content": message}
    #     new_user_message = {"role": "user", "content": "Give me 10 more real examples."}
        
    #     conversation.append(assitant_message)
    #     conversation.append(new_user_message)
        
    #     completion = client.chat.completions.create(
    #         model="gpt-4",
    #         messages=conversation
    #     )
        
    #     message = completion.choices[0].message.content
    
    #     print(message)
    
    #     data_augmentation_messages.append(message)


Give 10 examples of real news related to ESG (Environmental, Social, Governance) that has an impact duration less than 2 years. Each example should have a news title and news summary, connected with "||". Generate these examples in english language. Two example news:

    "EU Regulators Welcome, Critique New European Sustainability Reporting Standards || Europe’s three primary financial regulatory agencies, the European Supervisory Authorities (ESAs) each announced the release of their opinions on the first set of draft European Sustainability Reporting Standards, which set out the rules and requirements for companies to report on sustainability-related impacts, opportunities and risks under the EU’s upcoming Corporate Sustainable Reporting Directive (CSRD)."
    
    "Bank of America CEO: Sustainable Finance Demand Increasing Despite Macro Headwinds || Last year, the company reported that its 2021 sustainable finance activity reached an all-time high of $250 billion in capital mobiliz

In [62]:
augmented_dataframes = []

for idx, response in enumerate(data_augmentation_messages):
    augmented_news_list = response.split("\n\n")
    augmented_news_list = [i.split(". ", maxsplit=1)[1] for i in augmented_news_list]
    
    augmented_df = pd.DataFrame(augmented_news_list, columns=["sentence"])
    augmented_df["impact_length_idx"] = idx // 3
    
    augmented_dataframes.append(augmented_df)
    
augmented_df = pd.concat(augmented_dataframes).reset_index(drop=True)

In [63]:
augmented_df["url"] = "ChatGPT augmented with Bing Search"
augmented_df[["title", "content"]] = augmented_df["sentence"].str.split(" \|\| ", expand=True)
augmented_df["language"] = "English"

In [64]:
augmented_df

,sentence,impact_length_idx,url,title,content,language
0,"""Nestle Accelerates Net Zero Pledge with $3.6 ...",0,ChatGPT augmented with Bing Search,"""Nestle Accelerates Net Zero Pledge with $3.6 ...",The Swiss company announced it will spend arou...,English
1,"""Salesforce’s Work.Com Revamps for Post-Pandem...",0,ChatGPT augmented with Bing Search,"""Salesforce’s Work.Com Revamps for Post-Pandem...",Salesforce is debuting employee management and...,English
2,"""Amazon Faces Product Liability Lawsuits After...",0,ChatGPT augmented with Bing Search,"""Amazon Faces Product Liability Lawsuits After...",A US appeals court ruled that Amazon can be he...,English
3,"""Future Super Fills Multibillion-Dollar Green ...",0,ChatGPT augmented with Bing Search,"""Future Super Fills Multibillion-Dollar Green ...","Australian financial services provider, Future...",English
4,"""LinkedIn Invests in New Racial Justice Fund |...",0,ChatGPT augmented with Bing Search,"""LinkedIn Invests in New Racial Justice Fund",Microsoft-owned LinkedIn announced a new racia...,English
...,...,...,...,...,...,...
85,"""Uber Pledges to be Fully Electric by 2040 || ...",2,ChatGPT augmented with Bing Search,"""Uber Pledges to be Fully Electric by 2040",Ridesharing company Uber pledges to make its e...,English
86,"""Kellogg Company Targets 50% Reduction in Glob...",2,ChatGPT augmented with Bing Search,"""Kellogg Company Targets 50% Reduction in Glob...",Kellogg Company has set a goal to reduce green...,English
87,"""Tata Motors Aims to be Carbon Neutral by 2039...",2,ChatGPT augmented with Bing Search,"""Tata Motors Aims to be Carbon Neutral by 2039","Tata Motors, one of India's largest auto manuf...",English
88,"""Starbucks Commits to Achieve Resource Positiv...",2,ChatGPT augmented with Bing Search,"""Starbucks Commits to Achieve Resource Positiv...",None,English


In [65]:
augmented_df.to_parquet("../dataset/train_df_English_augmented.parquet")

# Use the Augmented df and relabel to ensure data quality

In [3]:
augmented_df = pd.read_parquet("../dataset/train_df_English_augmented.parquet")
 

In [5]:
augmented_df.head()

,sentence,impact_length_idx,url,title,content,language
0,"""Nestle Accelerates Net Zero Pledge with $3.6 ...",0,ChatGPT augmented with Bing Search,"""Nestle Accelerates Net Zero Pledge with $3.6 ...",The Swiss company announced it will spend arou...,English
1,"""Salesforce’s Work.Com Revamps for Post-Pandem...",0,ChatGPT augmented with Bing Search,"""Salesforce’s Work.Com Revamps for Post-Pandem...",Salesforce is debuting employee management and...,English
2,"""Amazon Faces Product Liability Lawsuits After...",0,ChatGPT augmented with Bing Search,"""Amazon Faces Product Liability Lawsuits After...",A US appeals court ruled that Amazon can be he...,English
3,"""Future Super Fills Multibillion-Dollar Green ...",0,ChatGPT augmented with Bing Search,"""Future Super Fills Multibillion-Dollar Green ...","Australian financial services provider, Future...",English
4,"""LinkedIn Invests in New Racial Justice Fund |...",0,ChatGPT augmented with Bing Search,"""LinkedIn Invests in New Racial Justice Fund",Microsoft-owned LinkedIn announced a new racia...,English


In [33]:
def generate_gpt_label(sentence):
        
    template = f"""Given the following news, output 0 if the ESG impact duration is below 2 years, 1 if the ESG impact duration is between 2 and 5 year and 2 if the ESG impact duration is more than 5 years. You only need to output the number, and do not need any further explanation.
    
    {sentence}"""
    
    conversation = [
            {"role": "system", "content": "You are an ESG analyst, skilled assessing the level and duration an event in the news article might have on the company"},
            {"role": "user", "content": template}
        ]
    
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=conversation,
        temperature=0.7
    )
    
    message = completion.choices[0].message.content
    
    return int(message)


augmented_df["gpt_label"] = augmented_df["sentence"].apply(generate_gpt_label)

In [34]:
augmented_df.groupby("gpt_label")["gpt_label"].count()

gpt_label
0     3
1    26
2    61
Name: gpt_label, dtype: int64

In [41]:
len(augmented_df[augmented_df["gpt_label"] == augmented_df["impact_length_idx"]])

38

In [48]:
def word_count(sentence):
    if sentence is not None:
        return len(sentence.split( ))
    return 0

In [49]:
augmented_df["content_wc"] = augmented_df["content"].apply(word_count)

In [50]:
augmented_df["content_wc"].describe()

count    90.000000
mean     35.788889
std      10.121170
min       0.000000
25%      30.250000
50%      34.500000
75%      41.000000
max      68.000000
Name: content_wc, dtype: float64